In [ ]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

<div class="namebox">    
Double Click to edit and enter your

1.  Name
2.  Student ID
3.  @ucr.edu email address
    
</div>

<div style=" font-size: 300% !important;
    margin-top: 1.5em;
    margin-bottom: 10px;
    font-weight: bold;
    line-height: 1.0;
    text-align:center;">
Programming Assignment 3: Join -- the parallelism
</div>

In this assignment, you'll learn about the concepts of:

1. Thread-level parallelism
2. Multi-threading programming

You are strongly encouraged to go through the following documents before starting.
1.  The x86-64 assembly http://www.cs.cmu.edu/~fp/courses/15213-s06/misc/asm64-handout.pdf
2.  Intel Alder Lake CPU Architectures https://ieeexplore.ieee.org/document/9747991
This assignment includes a programming assignment. 

Check the course schedule for due date(s).

We need to thank [Dr. Steven Swanson](https://cseweb.ucsd.edu/~swanson/) as a significant part of the assignment is orginated from Dr. Swanson's teaching materials.

# Programming Assignment

Your programming assignment in this assignment is an extension of your work on the assignment 5.  This time, you'll be parallelizing your implementation of matrix exponentiation.  You can use C++ threads, pthreads, or OpenMP.  The assignment assume you'll use OpenMP.  If you use one of the others, the course staff will be less able to help you out.

You are free to use your (and only your) solution to the previous assignment as a starting point for this assignment.

Most of this section is a verbatim copy of the PA for assignment 3.  The sections with new content have "(New for Assignment 5)" at the end of their names.

Obviously, the performance targets have been increased.  Roughly speaking, the targets have increased to 2.5x (4x on Gradescope servers)!!!  There are four cores on our machine, and from what we have learned, it's very conservative to expect that speedup!

## Performance Variability (New for Programming Assignment 3)

<div class="alert alert-block alert-danger">

**Multithreaded Performance is Variable**:  Multithreading introduces variability in performance.  You will only get credit for performance numbers recorded on gradescope, you may need to run your gradescope job more than once to get meaningful measurements.
    
</div>

Running code on multiple processors introduce performance variability -- some times up to 20% or more.  In the real world, you'd either tolerate this or spend a lot of time trying to fix it.  Neither of those works well in a assignment, because we have limited time and you need a grade that reflects the quality of your work rather than whether a run was lucky or unlucky.

To address this problem, the assignment includes does two things:
1.  It runs your code 10 times and takes the average.  This is implemented in the `Makefile`.
2.  It includes a "canary" program with known performance.  

The canary runs before your code and if the performance of the canary is too low, the autograder will reject the run.  This means that it is much more likely that your gradescope submissions will fail and you may need to submit several times to get a good measurement. 

With the canary filtering out slow runs, performance variation is less this 5%.  So there is some marginal value in submitting multiple times in the hopes of getting a slightly good score.

<div class="alert alert-block alert-danger">

**Budget time for multiple submissions**:  Having to resubmit to gradscope due to canary failure is totally predictable, and you have been warned.  Plan to submit (and resubmit as necessary) well-ahead of the deadline. 
    
</div>


## Reference Code (New for Programming Assignment 3)

The reference implementation is in `join_reference.hpp`.  It's basically the same as for Assignment 3 and implements the following SQL query.


``SELECT SUM(orders.quantity * products.price) AS value FROM orders INNER JOIN products WHERE ON orders.product_id = products.product_id GROUP BY products.brand;``


In [ ]:
render_code("join_reference.hpp")

Read through the code and comments to make sure you understand what the code is doing. 

## Detailed Requirements

The requirements for the assignment are pretty simple:

1. Values in ``orders`` and ``products`` can be any `int64_t` value.
2. Records in ``orders`` and ``products`` are sorted. You should read `join_main.cpp` carefully to know how are data sorted.
3. Your output must match the output of the code in `join_reference.hpp`.
4. Your implementation should go in `join_solution.hpp`.  The starter version is just a copy of `join_reference.hpp`.
5. Your implementation must generate exact the same result as `join_reference` function.
6. Your implementation must achieve at least 4 $\times$ speedup  on gradescope to score. The performance number you got in the datahub cluster does not count toward your final score for the programming assignment.


## Running the Code

The driver code for the assignment is in `join_main.cpp` and `join.cpp`.  `join_main.cpp` is mostly command line processing (take a look if you want).  `join.cpp` is what actually calls your code:

In [ ]:
render_code("join.cpp")

It defines two functions:

* `join_reference_c` implements the reference/baseline join/aggregation for the given SQL.
* `join_solution_c` calls your code.

To invoke these, you can build and run `join.exe`:

In [ ]:
!rm join.exe; rm -f build/join*; make join.exe

`join.exe` takes several command line parameters:

The notable ones are:

1. `-customers` -- number of total customers.
2. `-products` -- number of total products.
3. `-brands` -- number of total brands.
4. `-i` sets the number of iterations.
5. `-f` what functions to run.
6. `-o` sets where statistics should go.
7. `-v` compares the result with the reference solution.

The first three of these can take multiple values and `join.exe` will run all combinations and they will end up in `stats.csv`:

In [ ]:
!cs203 run "./join.exe -M 3300 -o stats.csv -v -t 1 4 8 -i 1 -customers 4096 -products 256 -brands 256 -f join_reference_c join_solution_c"
df =render_csv("stats.csv")
display_mono(df)

## OpenMP (New For Programming Assignment 3)

OpenMP is automatically turned for your code in this assignment, so the `#pragma` command we used for the histogram should work fine.

If you want to compile at the command line, you'll need to either do 

```
export OPENMP=yes
```

once each time you log in or invoke `make` like so:

```
make join.exe OPENMP=yes
```
each time you build.

### Key Commands

The three `#pragma`s you'll need for this assignment are 

1. `#pragma omp parallel for` for parallelizing loops.
2. `#pragma omp critical` for parallelizing loops.
3. `#pragma omp simd` for vectorizing loops

These are the only three used in the solution used to set the performance targets.

These three blog posts provide a good introduction to these commands:

* http://jakascorner.com/blog/2016/04/omp-introduction.html  
* http://jakascorner.com/blog/2016/05/omp-for.html
* http://jakascorner.com/blog/2016/07/omp-critical.html

They are required reading.

These articles provide some more advanced topics that might be useful:

* http://jakascorner.com/blog/2016/06/omp-for-scheduling.html
* http://jakascorner.com/blog/2016/06/omp-data-sharing-attributes.html
* http://jakascorner.com/blog/2016/07/omp-default-none-and-const.html

There's an enormous amount of bad information online about OpenMP.

### Looking at OpenMP Assembly

If you look at the assembly for OpenMP programs, you'll find that your loop body has been replaced with a function call.  OpenMP does this so it can tell it's worker threads to call the function to perform one iteration of your loop.

### Caveats for our Tools

First, `gprof` doesn't work on with multi-threaded programs.  You can use it for single-threaded runs, though.

Second, Moneta's cache model is not multithread-aware, so the cache hit/miss numbers for multithreaded programs are not meaningful.

Moneta may also show more threads that you might be expecting.  OpenMP threads seem to be Thread 0 and 13 and above.  If you see some threads that don't seem to be doing anything, that's not surprising or concerning.

Finally, our performance counting code only collects data for one thread.  For OpenMP code this is ok:  all the threads do basically the same thing.  But you'll notice, for instance, that if a loop runs in 4 threads, the measure instruction count will go down by ~1/4 (assuming multi-threading didn't add a lot of overhead).
 
 
### Controlling the Number of Threads

By default the autograder will run your code with 12 threads.  If you want to use a different number in your final run, you can call 

```			
omp_set_num_threads(thread_count);
```

in your function. You should call it before the first OpenMP `#pragma`.

I can control thread count during development with `-t`.  


## Things To Try

### Non-Deterministic Tests (New for Programming Assignment 3)

With threading, comes non-deterministic bugs.  This means that the tests may fail only occasionally for your code.  If this seems to be happening, a good strategy is to just run them repeatedly and confirm that it's the case.

It's not a bug in the benchmark, you have a thread synchronization error.


### General Tips (New for Programming Assignment 3)

In the examples we saw that loop tiling and OpenMP pragmas can work well together.  This carries through to how you should figure out what to parallelize.  It's worth your time to try parallelizing different loops and changing how your loops are nested to accommodate that.

A few tips:

1.  Use `ET` to guide your optimizations.
1.  At this points you have many tools avaiassignmentle to you -- `omp parallel for`, `omp simd`, compiler optimizations, tiling.  The number of combinations is enormous.   I suggest applying them in this order (from largest impact-per-effort to smallest):
    1.  Get last assignment into good shape (see notes above and slides from class)
    2.  `omp parallel for`
    3.  `omp simd`
    4.  Fiddling with other compiler options/per-function compiler options.
    5.  Crazy stuff like intrinsics for better SIMD performance.
1.  While tiling only applied to loops with reuse (because temporal locality requires reuse), `parallel for` can apply to loops without reuse.  Same for `omp simd`.
2.  `omp parallel for` implicitly does tiling, since it divides the iterations of the parallel loop across several cores.
1.  Nesting parallel for loops with OpenMP is not usually a good idea (although it should work).  Start by picking one loop to parallelize.
2.  You want to parallelize an outer loop, so that the threads are working on large pieces of computation and need to synchronize less.
3.  Pay close attention to whether iterations of your parallel loop are writing to the same locations.  If so, you'll need a `omp critical` to ensure correct updates.

This last point can be tricky.  If I have this code:

```
#pragma omp parallel for
for(int i = 0; i < 10; i++) {
    for(int j = 0; j < 10; j++) {
        for(int k = 0; k < 10; k++) {
            X[i][j] += Y[k][j];
```

The only store is the assignment to `X(i, j)`.  Since `i` is the index of the parallel loop, I know that no other thread will be updating `X(i,j)`, since no other thread will have the same value of `i`.

However, in this code:

```
#pragma omp parallel for
for(int i = 0; i < 10; i++) {
    for(int j = 0; j < 10; j++) {
        for(int k = 0; k < 10; k++) {
            X[k][j] += Y[i][j];
```

I don't have the same guarantee.  Since `i` does is not used to select an element in `X`, every other thread will write to that location as well.  In that case, I could do

```
#pragma omp parallel for
for(int i = 0; i < 10; i++) {
    for(int j = 0; j < 10; j++) {
        for(int k = 0; k < 10; k++) {
#pragma omp critical
            X[k][j] += Y[i][j];
```
Which will probably be really slow, or I could create a private tensor, do my updates there, and then merge them into `X`.

5.  Pay close attention to write sharing when you are deciding how to parallelize.  Can the iterations of your parallel loop iterations write to the same place?
4.  `omp simd` only works on inner loops.
5.  `omp parallel for` works best on outer loops.
7.  `gprof` doesn't work for multithreading, so use `ET` to measure how long things take.


### Useful C++ ( Partly New For Programming Assignment 3) 


#### Controlling Compiler Optimizations

First, you can prevent inlining of a particular function by declaring it like so:

```
void __attribute__((noinline)) join_solution(...)
```

This can make it easier to debug, because you can set a breakpoint on the function and it'll work like you expect.

Second, you can turn on arbitrary optimizations for particular functions like so:

```
#pragma GCC push_options
#pragma GCC optimize ("unroll-loops")

void your_function() {
}

#pragma GCC pop_options
```


#### Assertions

The `assert()` macro is useful tool for debugging and to avoid silly errors.

If you say

```
assert(a > b);
```

And the expression is not true at run time, the assert with "fail" your program will crash with a somewhat useful error message.

This is a useful way to document and enforce assumptions you make in your code.  For instance, I used an assert in `convolution_tiled_split()` to ensure that the tile size was > 8.

You can get access to  `assert()` with 

```
#include<cassert>
```

The overhead of asserts is low, but not zero.  I would not put any in one of your performance-critical loops.

If you want to include asserts in performance-critical areas, you can add `-DNDEBUG` to the optimizations in `config.make`.  It'll disable all the `assert()`s.



## Do Your Work Here

Below are the key commands you'll need to make progress on the assignment.

### Setting Optimization Flags

As in your last assignment, you can set optimization flags in `config.make`:

In [ ]:
render_code("config.make")

### Compiling and Running

You can compile and the benchmarks locally using this command.  This is only useful for debugging.  Performance running locally is not very meaningful:

In [ ]:
!make join.exe

Run the benchmark in the cloud and compare your performance with the reference.

In [ ]:
!make clean;make join.exe
!cs203 run './join.exe -M 3300 -i 1 -threads 1 4 8 -customers 1024 2048 -products 1024 2048 -brands 64 -f join_reference_c join_solution_c'

In [ ]:
df =render_csv("stats.csv")
display_mono(df)

## Final Measurement

Running the cell does just what the Gradescope autograder does.  And the cell below shows the name and target speedups for each benchmark.  This takes 1-2 minutes to run.


<div class="alert alert-block alert-danger">

**Only Gradescope Counts** The scores produced here **do not** count.  Only gradescope counts.  The results here should match what Gradescope does, but I would test your solution on Gradescope well-ahead of the deadline to ensure your code is working like you expect.
    
</div>

<div class="alert alert-block alert-danger">

**The autograder doesn't pass additional parameters**. You'll need to set up the optimal configurations your code in the best way possible.
    
</div>


In [ ]:
!rm -f build/join*
!cs203 run "make autograde"

And run the autograder

In [ ]:
!mkdir -p autograde; cp bench.csv autograde; cp correctness.csv autograde
!./autograde.py --submission autograde --results autograde.json
from autograde import compute_all_scores
df = compute_all_scores(dir="autograde")
display_mono(df)
print(f"total points (performance): {round(sum(df['capped_score']), 2)}")
display_mono(render_csv("correctness.csv"))
corrects = compute_correctness(dir="autograde")
print(f"correctness points: {corrects/4*100}")

The "capped_score" column contains the number of points you'll receive.

And see the autograder's output like this:

In [ ]:
render_code("autograde.json")

Most of it is internal stuff that gradscope needs, but the key parts are the `score`, `max_score`, and `output` fields.

All that's left is commit your code:

In [ ]:
!git commit -am "Solution to the assignment."
!git push

If `git push` asks for your username, you'll need to push from the command line.

If `git commit` tells you have uncommitted files, that's not a problem. 

If `git commit` tell you something like:

```
*** Please tell me who you are.

Run

git config --global user.email "you@example.com"
git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'prcheng@dsmlp-jupyter-prcheng.(none)')
Warning: Permanently added the RSA host key for IP address '140.82.112.3' to the list of known hosts.
Everything up-to-date
```

Then you can do (but fill in your @ucr.edu email and your name):

In [ ]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

# Recap

This assignment completes our tour of exploiting modern processor (multi-threaded processors) features.  It explored what's required to exploit thread-level parallelism. It shows examples achieving better TLPs is really difficult -- coherency, consistency, locks ...

# Turning in the programming assignment


You need to turn in your notebook and your programming assignment in the specific gradescope item.  
After you complete the assignment, you will turn it in by submitting your latest github repository.

**Step 1:**  Save your workbook!!!

In [ ]:
!for i in 1 2 3 4 5; do echo Save your files!!; sleep 1; done

**Step 2:**  Commit everything. Please run the following command.

In [ ]:
!git commit -am "Yay! I am ready to turn in!"
!git push

**Step 3**: 
Submit through gradescope
You'll turn in your programming assignment by providing gradescope with your github repo of this assignment.   It'll run the autograder and return the results.